# Detecting Defender Blocked by AppLocker

**AppLocker Defender Blocking** is a defense evasion technique where attackers abuse AppLocker policies to prevent Windows Defender (MsMpEng.exe) from executing, effectively disabling endpoint protection.

## The Attack Process

1. **Gain Administrative Access**: Attacker obtains local admin or domain admin privileges to modify AppLocker policies

2. **Create Deny Rule**: Configure AppLocker policy to explicitly block MsMpEng.exe (Windows Defender's main executable)

3. **Apply Policy**: Enforce the policy via Group Policy or local configuration

4. **Disable Protection**: Windows Defender is prevented from starting or is terminated, leaving the system unprotected

5. **Execute Malware**: With Defender disabled, attacker can execute malicious payloads without detection

## Why It's Effective

- **Legitimate Windows Feature**: AppLocker is a native security feature, making abuse less suspicious
- **Persistent**: Policy changes persist across reboots until removed
- **Low Visibility**: Many organizations don't monitor AppLocker block events
- **No Alert Generation**: Unlike tamper protection triggers, AppLocker blocks may not generate security alerts
- **Bypasses Tamper Protection**: Works even when Defender tamper protection is enabled

## Detection Indicators

AppLocker blocking signatures to monitor:

- **Event ID 8004**: AppLocker blocked a program from running
- **File Name**: MsMpEng.exe (Windows Defender main executable)
- **File Path**: Typically `C:\\Program Files\\Windows Defender\\MsMpEng.exe`
- **Policy Applied**: Check for new or modified AppLocker policies

### Example Detection Query
```kql
event.input.type:"winlog" AND 
event.code:8004 AND 
file.name:"MsMpEng.exe"
```

## Common Tools & Techniques

- **Group Policy Management Console (GPMC)** - Domain-wide policy deployment
- **Local Security Policy (secpol.msc)** - Local machine configuration
- **PowerShell** - Scripted policy modification via `Set-AppLockerPolicy`
- **Registry Modification** - Direct manipulation of AppLocker policies

## Related Defense Evasion Techniques

Attackers may combine AppLocker abuse with:
- **Service Manipulation** - Stopping Defender services
- **Registry Modification** - Disabling real-time protection
- **Credential Access** - Obtaining admin rights to modify policies

## Mitigation Strategies

1. **Monitor AppLocker Events**: Enable and centralize Event ID 8004 logging
2. **Restrict Policy Modifications**: Limit who can create/modify AppLocker policies
3. **Baseline AppLocker Rules**: Document legitimate rules and alert on deviations
4. **Defender Cloud Protection**: Enable cloud-delivered protection for additional telemetry
5. **EDR Solutions**: Deploy additional endpoint protection that can't be disabled by AppLocker
6. **Audit Policy Changes**: Monitor Group Policy modifications (Event IDs 4719, 4739)
7. **Least Privilege**: Minimize accounts with AppLocker modification rights

## References

- [MITRE ATT&CK: T1562.001 - Impair Defenses: Disable or Modify Tools](https://attack.mitre.org/techniques/T1562/001/)
- [Microsoft AppLocker Documentation](https://docs.microsoft.com/en-us/windows/security/threat-protection/windows-defender-application-control/applocker/applocker-overview)
- [Windows Defender Evasion Techniques](https://www.microsoft.com/security/blog/)

## Sigma Rule

## Elastic SIEM Query If running from Kibana

event.input.type:"winlog" AND event.code:8004 AND file.name:"MsMpEng.exe"

## Automated API Query for Elastic
- Make sure you are running Jupyter from venv

### Install Elastic and Tabulate

In [3]:
!pip3 install elasticsearch tabulate

ERROR! Session/line number was not unique in database. History logging moved to new session 10


### Return Elastic Results

In [2]:
import json
import warnings
from datetime import datetime, timedelta
from elasticsearch import Elasticsearch
from tabulate import tabulate
warnings.filterwarnings('ignore')

# Connection details - UPDATE THESE WITH YOUR ENVIRONMENT
ES_ENDPOINT = "https://192.168.138.203:9200"
API_KEY = "Wnoycm5wc0JXdzMtSUZidXZUNUc6U0xOVUhoaVNkOERtNEFzVmdRUXhjdw=="

# Index pattern
INDEX_PATTERN = "logs-*"

try:
    # Initialize Elasticsearch client
    client = Elasticsearch(
        [ES_ENDPOINT],
        api_key=API_KEY,
        verify_certs=False,
        ssl_show_warn=False
    )
    
    # Test connection
    if client.ping():
        print("✓ Successfully connected to Elasticsearch")
        info = client.info()
        print(f"✓ Cluster: {info['cluster_name']}")
        print(f"✓ Version: {info['version']['number']}\n")
    else:
        print("✗ Could not connect")
        exit(1)
    
    # Step 1: Test basic query for event.code 8004
    print("="*80)
    print("STEP 1: Testing basic query for event.code:8004 (AppLocker Block Events)")
    print("="*80)
    
    basic_query = {
        "query": {
            "term": {
                "event.code": "8004"
            }
        },
        "size": 1
    }
    
    response = client.search(index=INDEX_PATTERN, body=basic_query)
    print(f"Results with event.code as string '8004': {response['hits']['total']['value']}")
    
    # Try as integer
    basic_query["query"]["term"]["event.code"] = 8004
    response = client.search(index=INDEX_PATTERN, body=basic_query)
    print(f"Results with event.code as integer 8004: {response['hits']['total']['value']}")
    
    if response['hits']['total']['value'] > 0:
        print("\n✓ Found Event 8004 records! Here's a sample:")
        sample = response['hits']['hits'][0]['_source']
        
        # Print relevant fields to see their structure
        print("\n--- SAMPLE RECORD STRUCTURE ---")
        print(f"input.type: {sample.get('input', {}).get('type')}")
        print(f"event.code: {sample.get('event', {}).get('code')}")
        print(f"file.name: {sample.get('file', {}).get('name')}")
        print(f"file.path: {sample.get('file', {}).get('path')}")
        print(f"user.name: {sample.get('user', {}).get('name')}")
        print(f"host.name: {sample.get('host', {}).get('name')}")
        
        print("\n--- FULL EVENT DATA ---")
        print(json.dumps(sample, indent=2))
    else:
        print("❌ No Event 8004 records found. AppLocker may not be logging or no blocks have occurred.")
        print("   Note: AppLocker must be configured to generate audit/block events.")
        exit(1)
    
    # Step 2: Search for MsMpEng.exe blocks specifically
    print("\n" + "="*80)
    print("STEP 2: Searching for MsMpEng.exe blocks (Windows Defender)")
    print("="*80)
    
    defender_query = {
        "query": {
            "bool": {
                "must": [
                    {"term": {"event.code": "8004"}},
                    {"wildcard": {"file.name": "*MsMpEng.exe*"}}
                ]
            }
        },
        "size": 10,
        "sort": [{"@timestamp": {"order": "desc"}}]
    }
    
    response = client.search(index=INDEX_PATTERN, body=defender_query)
    print(f"Results: {response['hits']['total']['value']}")
    
    # Step 3: Try query_string (KQL-like syntax)
    print("\n" + "="*80)
    print("STEP 3: Testing with query_string (KQL-like)")
    print("="*80)
    
    query_string = {
        "query": {
            "query_string": {
                "query": 'event.input.type:"winlog" AND event.code:8004 AND file.name:"MsMpEng.exe"'
            }
        },
        "size": 10,
        "sort": [{"@timestamp": {"order": "desc"}}]
    }
    
    response = client.search(index=INDEX_PATTERN, body=query_string)
    print(f"Results: {response['hits']['total']['value']}")
    
    if response['hits']['total']['value'] > 0:
        print("\n⚠️  CRITICAL ALERT: Windows Defender blocked by AppLocker!")
        print("    This may indicate defense evasion activity.\n")
        
        # Display results in table format
        table_data = []
        for i, hit in enumerate(response['hits']['hits'], 1):
            source = hit['_source']
            row = {
                "#": i,
                "Timestamp": source.get('@timestamp', 'N/A'),
                "Host": source.get('host', {}).get('name', 'N/A'),
                "User": source.get('user', {}).get('name', 'N/A'),
                "File Path": source.get('file', {}).get('path', 'N/A'),
                "Policy Name": source.get('winlog', {}).get('event_data', {}).get('PolicyName', 'N/A')
            }
            table_data.append(row)
        
        print(tabulate(table_data, headers="keys", tablefmt="grid"))
        
        # Additional context
        print("\n--- INVESTIGATION STEPS ---")
        print("1. Verify if this is authorized testing or security validation")
        print("2. Check who modified AppLocker policies (Event IDs 4719, 4739)")
        print("3. Review related authentication events for the user account")
        print("4. Examine other defense evasion indicators on affected hosts")
        print("5. Validate current AppLocker policy configuration")
    else:
        print("\n✓ No Windows Defender blocks detected - Good!")
        print("  Continue monitoring for any suspicious AppLocker activity.")
    
    # Step 4: Check for other security product blocks
    print("\n" + "="*80)
    print("STEP 4: Checking for other security tool blocks by AppLocker")
    print("="*80)
    
    security_tools = [
        "MsMpEng.exe",      # Windows Defender
        "CSFalconService",  # CrowdStrike
        "cb.exe",           # Carbon Black
        "WMIC.exe",         # Windows Management Instrumentation (often blocked)
        "powershell.exe"    # PowerShell (common target)
    ]
    
    for tool in security_tools:
        tool_query = {
            "query": {
                "bool": {
                    "must": [
                        {"term": {"event.code": "8004"}},
                        {"wildcard": {"file.name": f"*{tool}*"}}
                    ]
                }
            },
            "size": 0  # Just get count
        }
        
        response = client.search(index=INDEX_PATTERN, body=tool_query)
        count = response['hits']['total']['value']
        if count > 0:
            print(f"  ⚠️  {tool}: {count} blocks detected")
        else:
            print(f"  ✓  {tool}: No blocks")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    if 'client' in locals():
        client.close()
        print("\n✓ Connection closed")

✓ Successfully connected to Elasticsearch
✓ Cluster: elastic-container-project
✓ Version: 9.2.2

STEP 1: Testing basic query for event.code:8004 (AppLocker Block Events)
Results with event.code as string '8004': 3
Results with event.code as integer 8004: 3

✓ Found Event 8004 records! Here's a sample:

--- SAMPLE RECORD STRUCTURE ---
input.type: winlog
event.code: 8004
file.name: MsMpEng.exe
file.path: None
user.name: SYSTEM
host.name: win11-host1

--- FULL EVENT DATA ---
{
  "agent": {
    "name": "Win11-Host1",
    "id": "5702fe82-d5fe-4e9d-a529-6d4a7608b0fb",
    "ephemeral_id": "2f0c9126-7dc5-42bc-9a30-23687048728a",
    "type": "filebeat",
    "version": "9.2.2"
  },
  "process": {
    "pid": 940
  },
  "winlog": {
    "computer_name": "Win11-Host1.hackinglab.com",
    "record_id": "269",
    "process": {
      "pid": 940,
      "thread": {
        "id": 7356
      }
    },
    "event_id": "8004",
    "task": "None",
    "provider_guid": "{CBDA4DBF-8D5D-4F69-9578-BE14AA540D22}",
 